Goal with this data is to

1.) Identify high-impact NAICS codes
    We can identify top industries by 
        # of establishments
        annual pay
        number of employees
        calculate LQ values by comparing to national data
            LQ values are (Regional share of industry employment) / (National share of industry employment) 

2.) With these 20-30 high-impact industries, we can look into zip-code level data and then plot (somehow) their distribution.
    This can help us identify patterns such as industry clusters and overlaps.
    Try to use geopandas for plotting

3.) Census data can also provide us MRF/waste disposal facilities (look into this some more)

4.) EPA data also provides some insight into MRFs (but I am not fully sure how they categorize them and select them) 
    More research will have to be done to cross compare Census data to EPA data

5.) Big question to figure out is is the amount of MRFs balanced regionally? (could we even figure this out with this data?)
    Are there any gaps in our current infrastructure?
    *** what is the nomenclature for MRF NAICS codes?

5.) Take the MAESTRI and ISDATA from european symbiosis examples, translate their NACE codes into NAICS codes using chatGPT
    Select high impact WA NAICS codes
        Using historical symbiosis connections 
        Find biggest opportunities in our state

*** there is an EIA dataset on energy consuption by industry, do we want to integrate this as well?
    high energy consumption could potentially make a company "higher impact"
    

In [20]:
import requests
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import seaborn as sns
from libpysal.weights import Queen
from esda.moran import Moran, Moran_Local
from esda.getisord import G_Local
import warnings
warnings.filterwarnings('ignore')


1.) Pull WA state manufacturing data and national manufacturing data from US County Buisiness Patterns dataset

In [ ]:
## retrieve national data
def get_national_manufacturing_data():
    """Get US national manufacturing data for Location Quotient calculations"""
    variables = ['NAME', 'NAICS2017', 'NAICS2017_LABEL', 'ESTAB', 'EMP']
    url = f"https://api.census.gov/data/2022/cbp?get={','.join(variables)}&for=us:1&NAICS2017=3*"
    
    try:
        response = requests.get(url)
        print(f"National data status: {response.status_code}")
        
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data[1:], columns=data[0])
            return df
        else:
            print("Failed to get national data")
            return None
    except Exception as e:
        print(f"Error getting national data: {e}")
        return None

In [ ]:
#retrieve washington data 
def get_wa_state_manufacturing_data():
    """Get Washington STATE-level manufacturing data (not county-level)"""
    variables = ['NAME', 'NAICS2017', 'NAICS2017_LABEL', 'ESTAB', 'EMP', 'PAYANN']
    # Changed from county:* to state:53 to get state-level aggregates
    url = f"https://api.census.gov/data/2022/cbp?get={','.join(variables)}&for=state:53&NAICS2017=3*"
    
    try:
        response = requests.get(url)
        print(f"WA state data status: {response.status_code}")
        
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data[1:], columns=data[0])
            print(f"Retrieved {len(df)} WA state-level records")
            return df
        else:
            print("Failed to get WA state data")
            return None
    except Exception as e:
        print(f"Error getting WA state data: {e}")
        return None

In [45]:
wa_data = get_wa_state_manufacturing_data()
wa_data = wa_data.loc[:, ~wa_data.columns.duplicated()]



WA state data status: 200
Retrieved 540 WA state-level records


In [ ]:
wa_data[['ESTAB', 'EMP']] = wa_data[['ESTAB', 'EMP']].apply(pd.to_numeric, errors='coerce')
print(wa_data)

wa_data['NAICS2017'] = wa_data['NAICS2017'].astype(str)
wa_data_3digit = wa_data[wa_data['NAICS2017'].str.len() == 3]
wa_data_4digit = wa_data[wa_data['NAICS2017'].str.len() == 4]
wa_data_6digit = wa_data[wa_data['NAICS2017'].str.len() == 6]

print('total establishments recorded by 3-digit NAICS', wa_data_3digit['ESTAB'].sum())
print('total establishments recorded by 4-digit NAICS', wa_data_4digit['ESTAB'].sum())
print('total establishments recorded by 6-digit NAICS', wa_data_6digit['ESTAB'].sum())


           NAME NAICS2017                                    NAICS2017_LABEL  \
0    Washington     31-33                                      Manufacturing   
1    Washington       311                                 Food manufacturing   
2    Washington      3111                          Animal food manufacturing   
3    Washington     31111                          Animal food manufacturing   
4    Washington    311111                     Dog and cat food manufacturing   
..          ...       ...                                                ...   
535  Washington    339950                                 Sign manufacturing   
536  Washington     33999              All other miscellaneous manufacturing   
537  Washington    339991  Gasket, packing, and sealing device manufacturing   
538  Washington    339992                   Musical instrument manufacturing   
539  Washington    339999              All other miscellaneous manufacturing   

     ESTAB     EMP    PAYANN state  
0 

will start off with analysis for 4-digit NAICS codes

In [28]:
print(wa_data_4digit)
print("Total Establishments:", wa_data_4digit['ESTAB'].sum())
print("Total Industries:", wa_data_4digit['NAICS2017'].nunique())

           NAME NAICS2017                                    NAICS2017_LABEL  \
2    Washington      3111                          Animal food manufacturing   
6    Washington      3112                          Grain and oilseed milling   
13   Washington      3113      Sugar and confectionery product manufacturing   
19   Washington      3114  Fruit and vegetable preserving and specialty f...   
26   Washington      3115                        Dairy product manufacturing   
..          ...       ...                                                ...   
498  Washington      3371  Household and institutional furniture and kitc...   
506  Washington      3372  Office furniture (including fixtures) manufact...   
512  Washington      3379      Other furniture related product manufacturing   
518  Washington      3391       Medical equipment and supplies manufacturing   
525  Washington      3399                  Other miscellaneous manufacturing   

     ESTAB    EMP  PAYANN state  
2    

In [ ]:
## identify most relevant sectors by # of establishments, # of employees, 
wa_data_top_est = wa_data_4digit.sort_values(by='ESTAB', ascending=False).head(20)
print(wa_data_top_est)

wa_data_top_emp = wa_data_4digit.sort_values(by='EMP', ascending=False).head(20)
print(wa_data_top_emp)



           NAME NAICS2017                                    NAICS2017_LABEL  \
67   Washington      3121                             Beverage manufacturing   
327  Washington      3327  Machine shops; turned product; and screw, nut,...   
169  Washington      3231            Printing and related support activities   
525  Washington      3399                  Other miscellaneous manufacturing   
42   Washington      3118                Bakeries and tortilla manufacturing   
304  Washington      3323  Architectural and structural metals manufacturing   
518  Washington      3391       Medical equipment and supplies manufacturing   
498  Washington      3371  Household and institutional furniture and kitc...   
140  Washington      3219                   Other wood product manufacturing   
221  Washington      3261                     Plastics product manufacturing   
258  Washington      3273          Cement and concrete product manufacturing   
418  Washington      3345  Navigational,

In [35]:
nat_data = get_national_manufacturing_data()

National data status: 200


In [ ]:
# basically complete same as above 
nat_data = nat_data.loc[:, ~nat_data.columns.duplicated()]

nat_data[['ESTAB', 'EMP']] = nat_data[['ESTAB', 'EMP']].apply(pd.to_numeric, errors='coerce')
print(nat_data)
print(nat_data['ESTAB'].sum())

nat_data['NAICS2017'] = nat_data['NAICS2017'].astype(str)
nat_data_3digit = nat_data[nat_data['NAICS2017'].str.len() == 3]
nat_data_4digit = nat_data[nat_data['NAICS2017'].str.len() == 4]
nat_data_6digit = nat_data[nat_data['NAICS2017'].str.len() == 6]

print('total establishments recorded by 3-digit NAICS', nat_data_3digit['ESTAB'].sum())
print('total establishments recorded by 4-digit NAICS', nat_data_4digit['ESTAB'].sum())
print('total establishments recorded by 6-digit NAICS', nat_data_6digit['ESTAB'].sum())

print(nat_data_4digit)
print("Total Establishments:", nat_data_4digit['ESTAB'].sum())
print("Total Industries:", nat_data_4digit['NAICS2017'].nunique())

              NAME NAICS2017                                  NAICS2017_LABEL  \
0    United States     31-33                                    Manufacturing   
1    United States       311                               Food manufacturing   
2    United States      3111                        Animal food manufacturing   
3    United States     31111                        Animal food manufacturing   
4    United States    311111                   Dog and cat food manufacturing   
..             ...       ...                                              ...   
643  United States    339992                 Musical instrument manufacturing   
644  United States    339993  Fastener, button, needle, and pin manufacturing   
645  United States    339994              Broom, brush, and mop manufacturing   
646  United States    339995                      Burial casket manufacturing   
647  United States    339999            All other miscellaneous manufacturing   

      ESTAB       EMP us  


In [38]:
def calculate_location_quotients(wa_data, national_data):
    """Calculate proper Location Quotients using national benchmarks"""
    
    print("\n=== Location Quotient Analysis ===")
    
    # Prepare Washington data - sum across all counties for each industry
    wa_by_industry = wa_data.groupby('NAICS2017')['EMP'].sum().reset_index()
    wa_total_emp = wa_by_industry['EMP'].sum()
    wa_by_industry['WA_SHARE'] = wa_by_industry['EMP'] / wa_total_emp
    
    print(f"Washington total manufacturing employment: {wa_total_emp:,}")
    
    # Prepare national data
    national_by_industry = national_data.groupby('NAICS2017')['EMP'].sum().reset_index()
    national_total_emp = national_by_industry['EMP'].sum()
    national_by_industry['NATIONAL_SHARE'] = national_by_industry['EMP'] / national_total_emp
    
    print(f"US total manufacturing employment: {national_total_emp:,}")
    
    # Merge and calculate Location Quotients
    lq_analysis = wa_by_industry.merge(
        national_by_industry[['NAICS2017', 'NATIONAL_SHARE']], 
        on='NAICS2017', 
        how='inner'
    )
    
    # Calculate LQ = (WA share) / (National share)
    lq_analysis['LOCATION_QUOTIENT'] = lq_analysis['WA_SHARE'] / lq_analysis['NATIONAL_SHARE']
    
    # Add industry labels
    industry_labels = wa_data[['NAICS2017', 'NAICS2017_LABEL']].drop_duplicates()
    lq_analysis = lq_analysis.merge(industry_labels, on='NAICS2017', how='left')
    
    # Find specializations (LQ > 1.0)
    specializations = lq_analysis[lq_analysis['LOCATION_QUOTIENT'] > 1.0].copy()
    specializations = specializations.sort_values('LOCATION_QUOTIENT', ascending=False)
    
    print(f"\nWashington Manufacturing Specializations (LQ > 1.0):")
    print(f"Found {len(specializations)} specialized industries")
    print("\nTop 15 Specializations:")
    
    for _, row in specializations.head(15).iterrows():
        print(f"• LQ {row['LOCATION_QUOTIENT']:>5.2f} | {row['NAICS2017_LABEL'][:55]:<55} | {row['EMP']:>8,} jobs")
    
    return lq_analysis, specializations

In [41]:
calculate_location_quotients(wa_data_4digit, nat_data_4digit)


=== Location Quotient Analysis ===
Washington total manufacturing employment: 256,710
US total manufacturing employment: 12,188,330

Washington Manufacturing Specializations (LQ > 1.0):
Found 22 specialized industries

Top 15 Specializations:
• LQ  8.21 | Seafood product preparation and packaging               |    5,041 jobs
• LQ  7.56 | Aerospace product and parts manufacturing               |   67,123 jobs
• LQ  2.96 | Sawmills and wood preservation                          |    5,587 jobs
• LQ  2.91 | Fruit and vegetable preserving and specialty food manuf |   10,010 jobs
• LQ  1.90 | Pulp, paper, and paperboard mills                       |    3,678 jobs
• LQ  1.62 | Beverage manufacturing                                  |    9,162 jobs
• LQ  1.53 | Industrial machinery manufacturing                      |    3,682 jobs
• LQ  1.45 | Pesticide, fertilizer, and other agricultural chemical  |    1,054 jobs
• LQ  1.32 | Semiconductor and other electronic component manufactur |    7,

(   NAICS2017    EMP  WA_SHARE  NATIONAL_SHARE  LOCATION_QUOTIENT  \
 0       3111    757  0.002949        0.004958           0.594715   
 1       3112    837  0.003260        0.004866           0.670037   
 2       3113    962  0.003747        0.007132           0.525438   
 3       3114  10010  0.038993        0.013404           2.909130   
 4       3115   1651  0.006431        0.013340           0.482105   
 ..       ...    ...       ...             ...                ...   
 78      3371   3963  0.015438        0.018873           0.817956   
 79      3372   1079  0.004203        0.008688           0.483779   
 80      3379    802  0.003124        0.002891           1.080752   
 81      3391   4032  0.015706        0.025000           0.628260   
 82      3399   4138  0.016119        0.020665           0.780044   
 
                                       NAICS2017_LABEL  
 0                           Animal food manufacturing  
 1                           Grain and oilseed milling  

2.) Now that we've identified top industries of industries of interest in terms of location quotients, we can plot clusters using zip-code level data.
Thinking to do a heatmap showing which of these industries are located.

Also want to look into these statistical techniques to demonstrate overlap between sectors geographically: 

Cross-correlation (which industries co-locate?), 
Moran's I (which industries cluster geographically?), 
LISA/G* (where are the clusters?)

In [ ]:
# was not able to figure out how to make API call for this so I downloaded data for all zip codes in WA
# this does not provide employee counts, but it shows # of establishments, which is what we need for mapping
# https://data.census.gov/table/CBP2022.CB2200CBP?q=CBP2022.CB2200CBP&g=040XX00US53$8610000
zip_data = pd.read_csv(r"C:\Users\masha\Downloads\ZIP_CBP2022.CB2200CBP_2025-06-20T014341\CBP2022.CB2200CBP-Data.csv")

In [ ]:
#select only 4-digit manufacturing NAICS codes
zip_data = zip_data.loc[:, ~zip_data.columns.duplicated()]
zip_data = zip_data[zip_data['EMPSZES_LABEL'] == 'All establishments']
print(zip_data)


In [ ]:
print(zip_data.columns)


In [ ]:
columns_to_keep = ['GEO_ID', 'NAME', 'NAICS2017', 'NAICS2017_LABEL', 'ESTAB']


In [ ]:
zip_data = zip_data[columns_to_keep]
zip_data['ZIP'] = zip_data['NAME'].str.extract(r'ZIP (\d{5})')
## keep only NAICS starting with 31, 32, 33 (manufacturing)
## keep only 4-digit NAICS
zip_data = zip_data[zip_data['NAICS2017'].str.startswith(('31', '32', '33'))]
zip_data = zip_data[zip_data['NAICS2017'].str.match(r'^\d{4}$')]

print(zip_data)

In [ ]:
zip_data['ESTAB'] = pd.to_numeric(zip_data['ESTAB'], errors='coerce')
print(zip_data['ESTAB'].sum())
#weird that sum of total establishments is different than for state-level dataset
#2741 for zip vs 6745 for state-level
#might need to look into data sources some more for this